# This notebook test the connection between the compute clusters and datalake

In [ ]:
# Import required PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Configuration for storage paths, catalog, and schemas
CONFIG = {
    'catalog_name': 'catalog_name',
    'bronze_schema': 'bronze_schema',
    'gold_schema': 'gold_schema',
    'storage_account': 'datalake name',
    'bronze_path': 'abfss://bronze@{storage_account}.dfs.core.windows.net/vitals',
    'gold_path': 'abfss://gold@{storage_account}.dfs.core.windows.net/output'
}

# Format storage paths using the storage account name
CONFIG['bronze_path'] = CONFIG['bronze_path'].format(storage_account=CONFIG['storage_account'])
CONFIG['gold_path'] = CONFIG['gold_path'].format(storage_account=CONFIG['storage_account'])

# Create a Spark session with Unity Catalog enabled
spark = SparkSession.builder \
    .appName("Azure Data Lake Access") \
    .getOrCreate()

try:
    # Set the catalog 
    print("Setting up Unity Catalog context...")
    spark.sql(f"USE CATALOG {CONFIG['catalog_name']}")
    
    # Use bronze schema
    print("\nUsing bronze schema for reading...")
    spark.sql(f"USE SCHEMA {CONFIG['bronze_schema']}")
    
    # Read using bronze schema
    print("Reading data from bronze container...")
    bronze_data = spark.read \
        .option("multiline", "true") \
        .option("inferSchema", "true") \
        .json(CONFIG['bronze_path']) \
        .limit(10)

    #bronze_data.cache()

    print("Sample data:")
    bronze_data.show()

    # Use gold schema
    print("\nUsing gold schema for writing...")
    spark.sql(f"USE SCHEMA {CONFIG['gold_schema']}")

    # Write using gold schema in delta lake, iceberg, or parquet
    bronze_data.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(f"{CONFIG['catalog_name']}.{CONFIG['gold_schema']}.patient_data")
   
    print("Write operation completed successfully")
       
except Exception as e:
    print(f"Error: {str(e)}")